In [ ]:
import glob, pylab, pandas as pd
import pydicom, numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pylab as plt
import os
import seaborn as sns
from imgaug import augmenters as iaa

In [ ]:
import keras
from keras import layers
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.utils import to_categorical, Sequence
import cv2
from keras import regularizers

In [ ]:
train_images_dir = '../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/'
# train_images = [f for f in listdir(train_images_dir) if isfile(join(train_images_dir, f))]

In [ ]:
     
!ls ../input

In [ ]:
print(listdir('../input/rsna-intracranial-hemorrhage-detection'))

In [ ]:
test_images_dir = '../input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/'
# test_images = [ f for f in listdir(test_images_dir) if isfile(join(test_images_dir,f))]

In [ ]:
train = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv')
train['Subtype'] = train['ID'].str.rsplit('_', 1, expand=True)[1]
train['ID'] = train['ID'].str.rsplit('_', 1, expand=True)[0]
train.head()

In [ ]:
train = pd.pivot_table(train, index="ID", columns="Subtype", values="Label" )
train.head()

test_data 

In [ ]:
test = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/stage_1_sample_submission.csv')
test.head()

In [ ]:
test['Subtype'] = test['ID'].str.rsplit('_', 1, expand=True)[1]
test['ID'] = test['ID'].str.rsplit('_', 1, expand=True)[0]
test.head()

In [ ]:
test = pd.pivot_table(test, index="ID", columns="Subtype", values="Label")
test.head()

**Healthy Patient & Diagnosed Patients**

In [ ]:
# sick_patients = train[train["any"]==1].index.values
# # print(len(sick_patients))
# # sick_patients = sick_patients[0:65536]
# #######################
# seed = 1

# healthy_patients = train[ train["any"] == 0 ].index.values
# healthy_patients_selection = np.random.RandomState(seed).choice(
#     healthy_patients, size=len(sick_patients), replace=False
# )

# selected_patients = list(set(healthy_patients_selection).union(set(sick_patients)))

# updated_train = train.loc[selected_patients].copy()
# len(healthy_patients_selection)
# updated_train.head()

In [ ]:
split_seed = 1
train_data, dev_data = train_test_split(train,
                                        test_size=0.3,
                                        stratify=train.values,
                                        random_state=split_seed)
print(train_data.shape)
print(dev_data.shape)

**DataGenerator**

In [ ]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm):
    window_center = 15
    window_width = 400 
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, path, list_IDs, labels, batch_size, img_size, img_channel, num_classes, shuffle=True):
        self.path = path
        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_channel = img_channel
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()
     
    
    def __len__(self):
        return int(np.floor(len(self.list_IDs)/self.batch_size))
    
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
#     def augment_img(self, image): 
#         augment_img = iaa.Sequential([
#             iaa.Crop(keep_size=True, percent=(0.01, 0.05), sample_independently=False),
#             iaa.Affine(rotate=(-10, 10)),
#             iaa.Fliplr(0.5)])
#         image_aug = augment_img.augment_image(image)
#         return image_aug
    
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    
#     def rescale_pixelarray(self, dataset):
#         image = dataset.pixel_array
#         rescaled_image = image * dataset.RescaleSlope + dataset.RescaleIntercept
#         rescaled_image[rescaled_image < -1024] = -1024
#         return rescaled_image

    
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, self.img_size, self.img_size))
        y = np.empty((self.batch_size, self.num_classes), dtype=np.float32)
        for i, ID in enumerate(list_IDs_temp):
            data_file = pydicom.dcmread(self.path+ ID +'.dcm')
#             img = self.rescale_pixelarray(data_file)
            img =  window_image(data_file)
            img = cv2.resize(img, (self.img_size, self.img_size))
#             img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
#             img = self.augment_img(img)
#             img = (img - np.min(img)) * (1 / (np.max(img) - np.min(img))) 
            X[i, ] = img
            y[i, ] = self.labels.loc[ID]
        X = np.repeat(X[..., np.newaxis], 1, -1)
#         X = X.astype('float32')
#         X -= X.mean(axis=0)
#         std = X.std(axis=0)
#         X /= X.std(axis=0)
        return X, y


**Test_data Generator**

In [ ]:
batch_size = 8
img_size = 256
img_channel = 1
num_classes = 6
train_generator = DataGenerator(train_images_dir, list(train_data.index), train_data,
                                batch_size, img_size, img_channel, num_classes)
val_generator = DataGenerator(train_images_dir, list(dev_data.index), dev_data,
                                batch_size, img_size, img_channel, num_classes)
test_generator = DataGenerator(test_images_dir, list(test.index), test,
                                batch_size, img_size, img_channel, num_classes)


**Vgg16 Model Architecture**

In [ ]:
model = Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu",input_shape=(256,256,1)))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation="relu",activity_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu",activity_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(0.2))
# model.add(Dense(128, input_dim=256,
#                 kernel_regularizer=regularizers.l2(0.0001),
#                 activity_regularizer=regularizers.l2(0.0001)))
# model.add(Dense(num_classes, activation="softmax"))
model.add(Dense(num_classes, activation="sigmoid"))
model.summary()

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input

In [ ]:
input_tensor = Input(shape=(224, 224, 3))

In [ ]:
base_model = InceptionV3(weights='../input/inception-v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False,input_tensor=input_tensor)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
print(base_model.input)
print(add_model(base_model.output))

In [ ]:
model.summary()

In [ ]:
from keras import backend as K

def weighted_log_loss(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights = np.array([0.25, 0.15, 0.15, 0.15, 0.15, 0.15])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    out = -(         y_true  * K.log(      y_pred) * class_weights
            + (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)
    
    return K.mean(out, axis=-1)

In [ ]:
Adam = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss="binary_crossentropy",
              optimizer=Adam, 
              metrics=[weighted_log_loss])

In [ ]:
history = model.fit_generator(generator=train_generator, validation_data=val_generator, epochs=1,max_queue_size=50,workers=50, use_multiprocessing=False)

In [ ]:
# from keras.models import load_model
# import h5py
# # model.save('my_model.h5')
# model = load_model('my_model.h5')

In [ ]:
batch_size = 5
test_generator = DataGenerator(test_images_dir, list(test.index), test,
                                batch_size, img_size, img_channel, num_classes) 

In [ ]:
predictions = model.predict_generator (test_generator, steps=15709, workers=10, use_multiprocessing=False, verbose=0)

In [ ]:
submission = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/stage_1_sample_submission.csv')
test_df = submission.ID.str.rsplit("_", 1, expand=True)
test_df = test_df.rename({0:"ID", 1:'subtype'}, axis=1)
test_df.loc[:, "Label"] = 0
test_df.head()

In [ ]:
def turn_pred_to_dataframe(test_df, predictions):
    df = pd.DataFrame(predictions, columns=test_df.columns, index=test_df.index)
    df = df.stack().reset_index()
    df.loc[:, 'ID'] = df.ID.str.cat(df.Subtype, sep="_")
    df = df.drop(["Subtype"], axis=1)
    df = df.rename({0:'Label'}, axis =1)
    return df

In [ ]:
testpred_df = turn_pred_to_dataframe(test, predictions )

In [ ]:
testpred_df.to_csv('submission.csv',index = False)

In [ ]:
testpred_df.head()

In [ ]:
# import os
# os.chdir(r'/kaggle/working')

In [ ]:
# testpred_df.to_csv(r'df_name.csv')

In [ ]:
# from IPython.display import FileLink
# FileLink(r'df_name.csv')